In [1]:
import time

In [2]:
#%%timeit -n 1 -r 1
start = time.perf_counter()

import torch

end = time.perf_counter()
print(f"Elapsed time: {end - start:.6f} seconds")

Elapsed time: 1.066151 seconds


In [3]:
import torch

In [2]:
torch.__version__

NameError: name 'torch' is not defined

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.get_device_name()

'NVIDIA L40S'

In [5]:
torch.cuda.list_gpu_processes()

'GPU:0\nno processes are running'

In [6]:
torch.cuda.device_count()

1

In [7]:
# See https://github.com/seafoodfry/ml-workspace/blob/main/gpu-sandbox/our-cuda-by-example/03-cudaDeviceProp.cu
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA L40S', major=8, minor=9, total_memory=45372MB, multi_processor_count=142)

In [8]:
# See https://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/
torch.cuda.get_arch_list()

['sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90']

In [9]:
# See
# https://pytorch.org/docs/stable/notes/cuda.html#optimizing-memory-usage-with-pytorch-cuda-alloc-conf
# and
# https://developer.nvidia.com/blog/using-cuda-stream-ordered-memory-allocator-part-1/
torch.cuda.get_allocator_backend()

'native'

In [10]:
# See https://pytorch.org/docs/stable/notes/mps.html
torch.backends.mps.is_available()

False

In [11]:
torch.backends.cuda.is_built()

True

In [12]:
torch.backends.cpu.get_cpu_capability()

'AVX2'

In [13]:
torch.tensor([1])

tensor([1])

In [14]:
torch.tensor([1, 2, 3])

tensor([1, 2, 3])

In [15]:
torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
])

tensor([[1, 2, 3],
        [4, 5, 6]])

In [16]:
# 3D.
torch.tensor([
    [
        [1, 2, 3],
        [4, 5, 6],
    ],
    [
        [7, 8, 9],
        [10, 11, 12],
    ]
])

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]]])

In [17]:
torch.tensor([1]).dtype

torch.int64

In [18]:
torch.tensor([1]).to(torch.float32).dtype

torch.float32

In [19]:
torch.tensor([1.0]).dtype

torch.float32